In [6]:
import json as j
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
json_data = None
with open('./yelp_academic_dataset_review.json') as data_file:
    lines = data_file.readlines()
    joined_lines = "[" + ",".join(lines) + "]"

    json_data = j.loads(joined_lines)

In [10]:
data = pd.DataFrame(json_data)

stemmer = SnowballStemmer('english')
words = stopwords.words("english")

data['cleaned'] = data['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

X_train, X_test, y_train, y_test = train_test_split(data['cleaned'], data.stars, test_size=0.2)



In [11]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [12]:
model = pipeline.fit(X_train, y_train)

In [13]:
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)


In [48]:


print("accuracy score: " + str(model.score(X_test, y_test)))

accuracy score: 0.5797920925579575


In [49]:
print(model.predict(['Dr. Merriman is very nice and explained everything to me.']))


[5]


In [25]:
print(model.predict(["Dr. Jain was extremely thorough with diagnosing our sons football injury. Although the outcome wasnt what we wanted to hear he was truthful and blunt about treatment. We appreciate his kindness| concern| and care he showed our son."]))

[5]


In [24]:
print(model.predict(['Exactly one year ago today| in a random act of kindness. Someone reached out to me and offered me the use of this scooter. After 3 days of juggling food| drinks| and crutches from my kitchen back to my bed (usually catastrophica']))


[1]


In [27]:
model.predict(['Exactly one year ago today| in a random act of kindness. Someone reached out to me and offered me the use of this scooter. After 3 days of juggling food| drinks| and crutches from my kitchen back to my bed (usually catastrophica'])[0]

1

In [28]:
model.predict(['Very satisfied and feel like I am in good hands.'])[0]

4

In [56]:
model.predict(["Sam is worst"])[0]

1

In [57]:
# 5 positive
# 4 somwat positive
# 3 negative
# 1<2
# 1 negative 


# worst
# bad
